In [2]:
from requests import get
import bs4
from bs4 import BeautifulSoup


import pandas as pd
import numpy as np

from time import sleep
from random import randint

from time import time
from warnings import warn

from datetime import datetime
from datetime import timedelta
from pytz import timezone

from IPython.display import clear_output


from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.common.exceptions import TimeoutException

import re
import os
import sys

import glob
import shutil
from pathlib import Path

In [116]:
# Path of the data folder
folder_data = r'C:\Users\motodangiuuuu\Crawler'

# Settings for web driver and Proxy
chrome_driver_path = os.path.join(os.getcwd(),'chromedriver.exe')
# PROXY = "socks5://localhost:9050" # IP:PORT or HOST:PORT

# create a new Chrome session
options = webdriver.ChromeOptions()
options.add_argument('--ignore-certificate-errors')
options.add_argument('--incognito')
# options.add_argument('--headless')
# options.add_argument('--proxy-server=%s' % PROXY)
driver = webdriver.Chrome(executable_path=chrome_driver_path, options=options)
driver.implicitly_wait(0)


In [118]:
url = 'https://careerbuilder.vn/vi/tim-viec-lam/e-commerce-manager-quan-ly-thuong-mai-dien-tu.35B59589.html'
driver.get(url)
sleep(3)

In [119]:
dict_detail = {}

In [120]:
element= WebDriverWait(driver, 20).until(EC.presence_of_element_located((By.CSS_SELECTOR, 'section[class="search-result-list-detail template-2"]')))
html_of_interest=driver.execute_script('return arguments[0].innerHTML',element)
soup_level1=BeautifulSoup(html_of_interest, 'lxml')
print("Block 1:", url)

In [121]:
#Tên việc làm
map_div = soup_level1.select('div[class="apply-now-content"] > div[class="job-desc"] > h1')
if map_div[0].text:
    dict_detail['Tên việc làm'] = map_div[0].text   
    
#Link   
dict_detail['Link'] = url

#Địa điểm   
map_div = soup_level1.select('div[class="detail-box"] > div[class="map"]')
dict_detail['Nơi làm việc'] = map_div[0].select('p > a')[0].text

In [121]:
#Ngày cập nhật   #Ngành nghề  #Hình thức    #Lương, Kinh nghiệm, Hết hạn nộp, Cấp bậc
map_div = soup_level1.select('div[class="col-lg-4 col-sm-6 item-blue"] > div[class="detail-box has-background"]')
for i in range(0, 2):
    for item in range(0, len(map_div[i].select('ul > li'))):
        dict_detail[map_div[i].select('ul > li')[item].select('li > strong')[0].text.strip()] = map_div[i].select('ul > li')[item].select('p')[0].text.strip()

#Phúc lợi    
for i in range(len(soup_level1.select('div[class="detail-row"]'))):
    attribute = soup_level1.select('div[class="detail-row"]')[i]
    if attribute.select('h3'):
        if attribute.select('h3')[0].text.strip() == 'Phúc lợi':
            phuc_loi=[]
            for i in attribute.select('ul> li'):
                phuc_loi.append(i.text.strip())
                dict_detail[attribute.select('h3')[0].text.strip()] = phuc_loi
        elif attribute.select('h3')[0].text.strip() == 'Thông tin khác':
            tt_khac=[]
            for i in attribute.select('ul> li'):
                tt_khac.append(i.text.strip())
                dict_detail[attribute.select('h3')[0].text.strip()] = tt_khac
        else:
            att =""
            for item in attribute.select('p, ul'):
                att = att + str(item) +("\n")
            att = att.replace("<", "[").replace(">", "]")
            dict_detail[attribute.select('h3')[0].text.strip()] = att
            
#Job Tags/Skills       
jobtags_skills=[]
tag_skills = soup_level1.select('div[class="job-tags"] > ul > li')
if tag_skills:
    for tag in tag_skills:
        jobtags_skills.append(tag.text)
dict_detail['Job tags / skills'] = jobtags_skills

# #Tổng quan công ty
link_company_info = soup_level1.select('a[class="employer job-company-name"]')[0]['href']
driver.get(link_company_info)
sleep(3)

element= WebDriverWait(driver, 20).until(EC.presence_of_element_located((By.CSS_SELECTOR, 'div[class="company-info"]')))
html_of_interest=driver.execute_script('return arguments[0].innerHTML',element)
soup_level1=BeautifulSoup(html_of_interest, 'lxml')  

company_info = soup_level1.select('div[class="content"]')[0]

name = company_info.select('p')[0].text
address = company_info.select('p')[1].text
dict_detail['Tên công ty'] = name
dict_detail['Địa chỉ'] = address

company_detail = company_info.select('ul > li')
for i in range(len(company_detail)):
    role = company_detail[i].text.split(":")[0].strip()
    detail = company_detail[i].text.split(":")[1].strip()
    dict_detail[role] = detail
 

Tên việc làm  \
0  E-COMMERCE MANAGER  (QUẢN LÝ THƯƠNG MẠI ĐIỆN TỬ)   

                                                Link Nơi làm việc  \
0  https://careerbuilder.vn/vi/tim-viec-lam/e-com...  Hồ Chí Minh   

  Ngày cập nhật                                         Ngành nghề  \
0    08/03/2021  Bán lẻ / Bán sỉ\n                             ...   

              Hình thức       Lương Kinh nghiệm  Cấp bậc Hết hạn nộp  \
0  Nhân viên chính thức  Cạnh tranh  5 - 10 Năm  Quản lý  06/04/2021   

                                            Phúc lợi  \
0  [Laptop, Chế độ bảo hiểm, Phụ cấp, Chế độ thưở...   

                                     Mô tả Công việc  \
0  [p][strong]Mô tả công việc:[/strong][/p]\n[ul]...   

                                   Yêu Cầu Công Việc  \
0  [p][strong]Yêu Cầu Công Việc:[/strong][/p]\n[u...   

                                      Thông tin khác Job tags / skills  
0  [Bằng cấp:\n                Đại học, Độ tuổi:\...                []

In [63]:
# python_button = driver.find_element_by_id('tabs-job-company') #button
# driver.execute_script("arguments[0].click();", python_button)

In [107]:
link_company_info = soup_level1.select('a[class="employer job-company-name"]')[0]['href']
driver.get(link_company_info)
sleep(3)

In [108]:
element= WebDriverWait(driver, 20).until(EC.presence_of_element_located((By.CSS_SELECTOR, 'div[class="company-info"]')))
html_of_interest=driver.execute_script('return arguments[0].innerHTML',element)
soup_level1=BeautifulSoup(html_of_interest, 'lxml')  

In [109]:
company_info = soup_level1.select('div[class="content"]')[0]

In [110]:
name = company_info.select('p')[0].text
address = company_info.select('p')[1].text
dict_detail['Tên công ty'] = name
dict_detail['Địa chỉ'] = address

In [111]:
company_detail = company_info.select('ul > li')
company_detail
for i in range(len(company_detail)):
    role = company_detail[i].text.split(":")[0].strip()
    detail = company_detail[i].text.split(":")[1].strip()
    dict_detail[role] = detail

In [122]:
df = pd.DataFrame.from_dict(dict_detail, orient='index')
df = df.transpose()
df

Tên việc làm  \
0  E-COMMERCE MANAGER  (QUẢN LÝ THƯƠNG MẠI ĐIỆN TỬ)   

                                                Link Nơi làm việc  \
0  https://careerbuilder.vn/vi/tim-viec-lam/e-com...  Hồ Chí Minh   

  Ngày cập nhật                                         Ngành nghề  \
0    08/03/2021  Bán lẻ / Bán sỉ\n                             ...   

              Hình thức       Lương Kinh nghiệm  Cấp bậc Hết hạn nộp  \
0  Nhân viên chính thức  Cạnh tranh  5 - 10 Năm  Quản lý  06/04/2021   

                                            Phúc lợi  \
0  [Laptop, Chế độ bảo hiểm, Phụ cấp, Chế độ thưở...   

                                     Mô tả Công việc  \
0  [p][strong]Mô tả công việc:[/strong][/p]\n[ul]...   

                                   Yêu Cầu Công Việc  \
0  [p][strong]Yêu Cầu Công Việc:[/strong][/p]\n[u...   

                                      Thông tin khác Job tags / skills  \
0  [Bằng cấp:\n                Đại học, Độ tuổi:\...                []   

                                         Tên công ty  \
0  Công Ty Cổ Phần Đầu Tư Thương Mại Quốc Tế Mặt ...   

                                             Địa chỉ Qui mô công ty  \
0  Tầng 25 tòa nhà Handico, Số 3 Mễ Trì Hạ, Nam T...           5000   

  Loại hình hoạt động                Website  
0             Cổ phần  www.redsun-iti.com.vn

In [123]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1 entries, 0 to 0
Data columns (total 20 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   Tên việc làm         1 non-null      object
 1   Link                 1 non-null      object
 2   Nơi làm việc         1 non-null      object
 3   Ngày cập nhật        1 non-null      object
 4   Ngành nghề           1 non-null      object
 5   Hình thức            1 non-null      object
 6   Lương                1 non-null      object
 7   Kinh nghiệm          1 non-null      object
 8   Cấp bậc              1 non-null      object
 9   Hết hạn nộp          1 non-null      object
 10  Phúc lợi             1 non-null      object
 11  Mô tả Công việc      1 non-null      object
 12  Yêu Cầu Công Việc    1 non-null      object
 13  Thông tin khác       1 non-null      object
 14  Job tags / skills    1 non-null      object
 15  Tên công ty          1 non-null      object
 16  Địa chỉ     